In [1]:
import chromadb
import pandas as pd
from pathlib import Path
from IPython.display import HTML
from sentence_transformers import SentenceTransformer

In [2]:
import sys
sys.path.append('../../src')
from DataLoader import DataLoader

## Vorbereitung

### Laden der vorbereitenten Vector-Datenbank

In [3]:
chroma_file = '/media/sz/Data/Connected_Lecturers/Opal_crosslab/processed/chroma_db'

chroma_client = chromadb.PersistentClient(path=chroma_file)
collection = chroma_client.get_or_create_collection(
            name="oer_connected_lecturer"
)

### Laden des AI generierten Datensatzes 

In [4]:
data_folder =  "/media/sz/Data/Connected_Lecturers/Opal_crosslab/raw"
data_ai = "OPAL_ai_meta.p"

df_ai = pd.read_pickle(Path(data_folder) / data_ai)
df_ai["filename"]=df_ai["pipe:ID"]+"."+df_ai["pipe:file_type"]
df_ai.drop(columns=["ai:keywords_gen","ai:keywords_dnb"],inplace=True)
df_ai.head(3)


,pipe:ID,pipe:file_type,ai:author,ai:revisedAuthor,ai:affilation,ai:title,ai:type,ai:keywords_ext,ai:dewey,ai:keywords_ext_ids,...,ai:keywords_ext_ids_valid,ai:keywords_ext_valid_ratio,ai:keywords_dnb_ids_valid,ai:keywords_dnb_valid_ratio,ai:keywords_gen_ids_valid,ai:keywords_gen_valid_ratio,all_ddc_valid_ids,ddc_valid_count,valid_ddc_keywords,filename
0,8I6sM5zapD60,pdf,"Stephan Gerhold, Marcel Beyer",[Vorname='Stephan' Familienname='Gerhold' Tite...,,Übung 3 Photogrammetrie,Aufgabenblatt,"Photogrammetrie, digitale Entzerrung, Resampli...",,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 18602, 14]",...,"[0, 1, 4, 14]",0.333333,"[0, 1, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21,...",0.9375,"[0, 1, 4, 11, 12, 14]",0.545455,"[0, 1, 4, 14, 0, 1, 11, 12, 14, 15, 16, 17, 18...",25,"[Photogrammetrie, digitale Entzerrung, Paralle...",8I6sM5zapD60.pdf
1,8ZICOHBmAHyQ,pdf,"(Leerstring, da kein Autor im Dokument erwähnt...",[],,3.1 Grundlagen der Tragwerke,Skript,"Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...",,"[32, 33, 34, 10180, 25, 26, 27, 28, 29, 30, 31]",...,"[33, 34, 25, 30]",0.363636,"[40, 41, 43, 44, 45, 46, 47, 50, 51, 25, 30]",0.6875,"[33, 36, 39, 25]",0.363636,"[33, 34, 25, 30, 40, 41, 43, 44, 45, 46, 47, 5...",19,"[Freiheitsgrade, Gelenke, Tragwerke, Festigkei...",8ZICOHBmAHyQ.pdf
2,8Lfz8SAKa6k0,pdf,Robert Ringel,[Vorname='Robert' Familienname='Ringel' Titel=''],,Gedanken und Notizen zu unserem Mini-KI-Worksh...,Protokoll,"Künstliche Intelligenz, Automatisierung, Indus...",,"[64, 58, 52, 53, 54, 55, 56, 57, 3706, 59, 60,...",...,"[58, 52, 53, 54, 55, 56, 3706, 59, 60]",0.750000,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 52, 5...",0.9375,"[65, 58, 52, 53, 56, 3706, 60, 62]",0.666667,"[58, 52, 53, 54, 55, 56, 3706, 59, 60, 67, 68,...",32,"[Chat-GPT, Künstliche Intelligenz, Automatisie...",8Lfz8SAKa6k0.pdf


In [5]:
files = "OPAL_files_attrib.p"

df_files = pd.read_pickle(Path(data_folder) / files)
df_files.drop(columns=["opal:comment", "opal:language", "opal:publicationMonth", "opal:publicationYear"],inplace=True)
df_files.head(3)

,opal:filename,opal:oer_permalink,opal:license,opal:author,opal:title,opal:revisedAuthor,pipe:ID,pipe:file_type
0,SC_Hardware.png,https://bildungsportal.sachsen.de/opal/oer/6TI...,CC BY 4.0 Int.,Daniela Dobeleit,Hardware zum Styroporschneiden,[Vorname='Daniela' Familienname='Dobeleit' Tit...,6TIlFGfTxXkw,png
1,beleg.pdf,https://bildungsportal.sachsen.de/opal/oer/8I6...,CC BY-SA 4.0 Int.,,,,8I6sM5zapD60,pdf
2,Bereich III.pdf,https://bildungsportal.sachsen.de/opal/oer/8ZI...,CC BY-NC-ND 4.0 Int.,,,,8ZICOHBmAHyQ,pdf


In [6]:
df_ai = df_ai.merge(df_files[['pipe:ID', "opal:oer_permalink"]], on="pipe:ID", how="left")
df_ai.drop(columns=["pipe:ID", "pipe:file_type"],inplace=True)
df_ai.head(3)

,ai:author,ai:revisedAuthor,ai:affilation,ai:title,ai:type,ai:keywords_ext,ai:dewey,ai:keywords_ext_ids,ai:keywords_gen_ids,ai:keywords_dnb_ids,...,ai:keywords_ext_valid_ratio,ai:keywords_dnb_ids_valid,ai:keywords_dnb_valid_ratio,ai:keywords_gen_ids_valid,ai:keywords_gen_valid_ratio,all_ddc_valid_ids,ddc_valid_count,valid_ddc_keywords,filename,opal:oer_permalink
0,"Stephan Gerhold, Marcel Beyer",[Vorname='Stephan' Familienname='Gerhold' Tite...,,Übung 3 Photogrammetrie,Aufgabenblatt,"Photogrammetrie, digitale Entzerrung, Resampli...",,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 18602, 14]","[0, 1, 2, 3, 4, 9, 10, 11, 12, 13, 14]","[0, 1, 9, 11, 12, 14, 15, 16, 17, 18, 19, 20, ...",...,0.333333,"[0, 1, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21,...",0.9375,"[0, 1, 4, 11, 12, 14]",0.545455,"[0, 1, 4, 14, 0, 1, 11, 12, 14, 15, 16, 17, 18...",25,"[Photogrammetrie, digitale Entzerrung, Paralle...",8I6sM5zapD60.pdf,https://bildungsportal.sachsen.de/opal/oer/8I6...
1,"(Leerstring, da kein Autor im Dokument erwähnt...",[],,3.1 Grundlagen der Tragwerke,Skript,"Tragwerke, Lagerreaktionen, Gleichgewichtsbedi...",,"[32, 33, 34, 10180, 25, 26, 27, 28, 29, 30, 31]","[33, 35, 10180, 36, 37, 38, 39, 25, 26, 27, 28]","[10180, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49...",...,0.363636,"[40, 41, 43, 44, 45, 46, 47, 50, 51, 25, 30]",0.6875,"[33, 36, 39, 25]",0.363636,"[33, 34, 25, 30, 40, 41, 43, 44, 45, 46, 47, 5...",19,"[Freiheitsgrade, Gelenke, Tragwerke, Festigkei...",8ZICOHBmAHyQ.pdf,https://bildungsportal.sachsen.de/opal/oer/8ZI...
2,Robert Ringel,[Vorname='Robert' Familienname='Ringel' Titel=''],,Gedanken und Notizen zu unserem Mini-KI-Worksh...,Protokoll,"Künstliche Intelligenz, Automatisierung, Indus...",,"[64, 58, 52, 53, 54, 55, 56, 57, 3706, 59, 60,...","[64, 65, 66, 58, 52, 53, 56, 57, 3706, 60, 62,...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 5...",...,0.750000,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 52, 5...",0.9375,"[65, 58, 52, 53, 56, 3706, 60, 62]",0.666667,"[58, 52, 53, 54, 55, 56, 3706, 59, 60, 67, 68,...",32,"[Chat-GPT, Künstliche Intelligenz, Automatisie...",8Lfz8SAKa6k0.pdf,https://bildungsportal.sachsen.de/opal/oer/8Lf...


## Suchanfragen

### Textbasiert in den Chunks der Datenbank

In [7]:
suchbegriff = "Transformator"

results = collection.get(where_document={"$contains": suchbegriff})


In [8]:
relevant = pd.DataFrame(results["metadatas"]).groupby("filename").agg({"page":"unique"})\
                                  .reset_index()\
                                  .merge(df_ai, left_on="filename", right_on="filename", how="left")\
                                  .dropna(subset=["opal:oer_permalink"])\
                                  [["filename", "ai:title", "ai:revisedAuthor", "ai:affilation", "ai:dewey",
                                   "opal:oer_permalink","page"]]\
                                  .head(15)

HTML(relevant.to_html(render_links=True, escape=False))

,filename,ai:title,ai:revisedAuthor,ai:affilation,ai:dewey,opal:oer_permalink,page
0,10XV5OePU7suY.pdf,EMA-LAB-10XV5OePU7suY,[],,,https://bildungsportal.sachsen.de/opal/oer/10XV5OePU7suY,"[12, 31]"
1,10XqEI89gsBlI.pdf,EMA-LAB-131 - Versuchsprotokoll Drehstromasynchronmaschine mit Schleifringläufer,[],,,https://bildungsportal.sachsen.de/opal/oer/10XqEI89gsBlI,"[12, 31]"
2,11317Id_pG4R4.pdf,Fakultät Physik - Physikdidaktik Portfolio ZUM SEMINAR VERTIEFUNG DER PHYSIKDIDAKTIK,"[Vorname='Falk' Familienname='Beuchel' Titel='', Vorname='Kerstin' Familienname='Gedigk' Titel='', Vorname='Josef' Familienname='Leisen' Titel='']",Oberschule Rödertal,,https://bildungsportal.sachsen.de/opal/oer/11317Id_pG4R4,[22]
3,11DjIJ7kvckfs.pdf,Grundlagen Sft: 8. Werkst. Bauw. 8.3. Alu.-Legierungen,[],TU Berlin,,https://bildungsportal.sachsen.de/opal/oer/11DjIJ7kvckfs,[20]
4,11HCuNyCVZVoQ.pdf,2. Werkstoffe und Bauweisen,"[Vorname='Werner' Familienname='Steinke' Titel='', Vorname='Schraut' Familienname='Leutenegger' Titel='', Vorname='Ganzer' Familienname='' Titel='']",,,https://bildungsportal.sachsen.de/opal/oer/11HCuNyCVZVoQ,[12]
5,11KHAOaGHYJwM.pdf,2. Werkstoffe und Bauweisen,"[Vorname='' Familienname='Havel' Titel='Havel', Vorname='' Familienname='Ganzer' Titel='Ganzer', Vorname='Werner' Familienname='Steinke' Titel='Werner Steinke', Vorname='' Familienname='Schraut' Titel='Schraut', Vorname='' Familienname='Leutenegger' Titel='Leutenegger', Vorname='' Familienname='Hassel' Titel='Hassel', Vorname='' Familienname='Ehrenstein' Titel='Ehrenstein']",,,https://bildungsportal.sachsen.de/opal/oer/11KHAOaGHYJwM,[12]
6,12-z6JEPV0n80.pdf,2. Werkstoffe und Bauweisen,"[Vorname='Havel' Familienname='' Titel='', Vorname='Taschinger' Familienname='' Titel='', Vorname='Werner' Familienname='Steinke' Titel='', Vorname='Ganzer' Familienname='' Titel='', Vorname='Schraut' Familienname='' Titel='', Vorname='Leutenegger' Familienname='' Titel='']",,,https://bildungsportal.sachsen.de/opal/oer/12-z6JEPV0n80,[12]
7,12X_RPO6yOouo.pdf,Bremsen und Bremsysteme für Schienenfahrzeuge,[],,,https://bildungsportal.sachsen.de/opal/oer/12X_RPO6yOouo,[20]
8,13CHB1bBHXX_k.pdf,Fahrzeugtechnik - Antriebskräfte und Zugkraftübertragung bei Schienenfahrzeugen,[],,,https://bildungsportal.sachsen.de/opal/oer/13CHB1bBHXX_k,[51]
9,17eBg-8vq8z3w.pdf,EÜV Lernpaket 7 Ohmsch-Induktives Systemelement,[Vorname='Wolfgang' Familienname='Schufft' Titel='Prof. Dr.-Ing.'],Fakultät für Elektrotechnik und Informationstechnik,,https://bildungsportal.sachsen.de/opal/oer/17eBg-8vq8z3w,[1]


### Embeddings-Basiert

In [9]:
model = SentenceTransformer("jinaai/jina-embeddings-v2-base-de", trust_remote_code=True)

In [10]:
document = """
   Transformatoren sind elektrische Betriebsmittel, die elektrische Energie von einer Spannungsebene auf eine andere übertragen. 
   Sie bestehen aus einem Eisenkern und mindestens zwei galvanisch getrennten Wicklungen (Primär- und Sekundärwicklung).
   Transformatoren arbeiten nach dem Prinzip der elektromagnetischen Induktion und werden zur Spannungstransformation 
   in elektrischen Energieversorgungsnetzen eingesetzt.
   """

embedding = model.encode(document)

In [11]:
results = collection.query(
    query_embeddings=[embedding],  # Anstatt query_texts
    n_results=30
)

In [12]:
relevant_ = pd.DataFrame(results["metadatas"][0]).groupby("filename").agg({"page":"unique"})\
                                  .reset_index()\
                                  .merge(df_ai, left_on="filename", right_on="filename", how="left")\
                                  .dropna(subset=["opal:oer_permalink"])\
                                  [["filename", "ai:title", "ai:revisedAuthor", "ai:affilation", "ai:dewey",
                                   "ai:dewey", "opal:oer_permalink","page"]]\
                                  .head(30)

HTML(relevant_.to_html(render_links=True, escape=False))

,filename,ai:title,ai:revisedAuthor,ai:affilation,ai:dewey,ai:dewey,opal:oer_permalink,page
0,129BoBFjVmFKQ.pdf,EMA-LAB Projektbericht,[],,,,https://bildungsportal.sachsen.de/opal/oer/129BoBFjVmFKQ,"[18, 26, 42, 36]"
1,13CHB1bBHXX_k.pdf,Fahrzeugtechnik - Antriebskräfte und Zugkraftübertragung bei Schienenfahrzeugen,[],,,,https://bildungsportal.sachsen.de/opal/oer/13CHB1bBHXX_k,[43]
2,17c-TeiZ-rXRI.pdf,Die Geschichte des Computers,[],,,,https://bildungsportal.sachsen.de/opal/oer/17c-TeiZ-rXRI,[14]
3,30QkY3VKPdio.pdf,Arbeitsblatt für die bibliographische Katalogisierung und Indexierung,"[Vorname='Fricke' Familienname='' Titel='', Vorname='Löffler' Familienname='' Titel='', Vorname='Benabdellah' Familienname='' Titel='', Vorname='Kache' Familienname='' Titel='']",,,,https://bildungsportal.sachsen.de/opal/oer/30QkY3VKPdio,[12]
4,3EzCjcsplOFE.pdf,6. Antriebskrafte,[],,,,https://bildungsportal.sachsen.de/opal/oer/3EzCjcsplOFE,[51]
5,3jCsQQlEluOU.pdf,Technische Universität Dresden Fakultät Erziehungswissenschaften Institut für Berufspädagogik und Berufliche Didaktik Wintersemester 2019/2020 Prüfungsleistung Modul Spezifische Fachdidaktik Gestaltung von Lernumgebungen Skizze einer Unterrichtsidee im Fach WTHS Thema: Grundschaltungen der Beleuchtungstechnik am Beispiel der Wechselschaltung,"[Vorname='Silvio' Familienname='Kraschewski' Titel='Silvio Kraschewski', Vorname='Daniela' Familienname='Dobeleit' Titel='Daniela Dobeleit', Vorname='Julius' Familienname='Kuhl' Titel='Julius Kuhl', Vorname='U' Familienname='Rudolph' Titel='U Rudolph']",Technische Universität Dresden,,,https://bildungsportal.sachsen.de/opal/oer/3jCsQQlEluOU,[10]
6,3uNzUd25x4Ho.pdf,6. Antriebskrafte,"[Vorname='Martin' Familienname='Kache' Titel='Dr.-Ing.', Vorname='Tobias' Familienname='Bregulla' Titel='Dipl.-Ing.']",Professur für Elektrische Bahnen,,,https://bildungsportal.sachsen.de/opal/oer/3uNzUd25x4Ho,[51]
7,46zKmzpp5m44.pdf,Vorlesung Triebfahrzeugtechnik (Antriebskonfigurationen) Professur für Technik spurgeführter Fahrzeuge Dr.-Ing. Martin Kache Sommersemester 2021,[Vorname='Martin' Familienname='Kache' Titel='Dr.-Ing.'],TU Dresden,,,https://bildungsportal.sachsen.de/opal/oer/46zKmzpp5m44,[79]
8,4NlrbBrv_hkE.pdf,EMA-ELT04070 - Bibliographische Katalogisierung und Indexierung,[],,,,https://bildungsportal.sachsen.de/opal/oer/4NlrbBrv_hkE,[23]
9,4c_VRiKjzy3U.pdf,EMA Arbeitsblätter,[],,,,https://bildungsportal.sachsen.de/opal/oer/4c_VRiKjzy3U,[21]


## Systematisches Testen der Vektordatenbank

### Teststrategie:
1. **Fachbereichs-Tests**: Verschiedene Disziplinen (Elektrotechnik, Informatik, Maschinenbau, etc.)
2. **Granularitäts-Tests**: Spezifische vs. allgemeine Begriffe
3. **Sprachstil-Tests**: Fachsprache vs. Umgangssprache
4. **Konzept-Tests**: Theoretische vs. praktische Anfragen
5. **Vergleichstests**: Embedding vs. Keyword-Suche

In [13]:
def test_query(query_text, test_name="", n_results=10, show_details=True):
    """
    Testet eine Suchanfrage sowohl mit Embedding als auch mit Keyword-Suche
    """
    print(f"\n{'='*60}")
    print(f"TEST: {test_name}")
    print(f"Query: {query_text}")
    print(f"{'='*60}")
    
    # 1. Embedding-basierte Suche
    embedding = model.encode(query_text)
    embedding_results = collection.query(
        query_embeddings=[embedding],
        n_results=n_results
    )
    
    # 2. Keyword-basierte Suche (erstes Wort als Beispiel)
    first_word = query_text.split()[0]
    keyword_results = collection.get(where_document={"$contains": first_word})
    
    # Verarbeite Embedding-Ergebnisse
    if embedding_results["metadatas"] and embedding_results["metadatas"][0]:
        embedding_df = pd.DataFrame(embedding_results["metadatas"][0])\
                         .groupby("filename").agg({"page":"unique"})\
                         .reset_index()\
                         .merge(df_ai, left_on="filename", right_on="filename", how="left")\
                         .dropna(subset=["opal:oer_permalink"])
        
        print(f"\n🔍 EMBEDDING-SUCHE: {len(embedding_df)} relevante Dokumente gefunden")
        if show_details and len(embedding_df) > 0:
            top_results = embedding_df[["filename", "ai:title", "ai:dewey"]].head(5)
            for idx, row in top_results.iterrows():
                print(f"  📄 {row['ai:title'][:50]}... (DDC: {row['ai:dewey']})")
    else:
        print("🔍 EMBEDDING-SUCHE: Keine Ergebnisse gefunden")
    
    # Verarbeite Keyword-Ergebnisse
    if keyword_results["metadatas"]:
        keyword_df = pd.DataFrame(keyword_results["metadatas"])\
                       .groupby("filename").agg({"page":"unique"})\
                       .reset_index()\
                       .merge(df_ai, left_on="filename", right_on="filename", how="left")\
                       .dropna(subset=["opal:oer_permalink"])
        
        print(f"\n🔤 KEYWORD-SUCHE '{first_word}': {len(keyword_df)} relevante Dokumente gefunden")
        if show_details and len(keyword_df) > 0:
            top_results = keyword_df[["filename", "ai:title", "ai:dewey"]].head(3)
            for idx, row in top_results.iterrows():
                print(f"  📄 {row['ai:title'][:50]}... (DDC: {row['ai:dewey']})")
    else:
        print(f"🔤 KEYWORD-SUCHE '{first_word}': Keine Ergebnisse gefunden")
    
    return embedding_df if 'embedding_df' in locals() else None, keyword_df if 'keyword_df' in locals() else None

### 1. Fachbereichs-Tests

In [14]:
# Test verschiedener Fachbereiche
fachbereich_tests = [
    ("Algorithmen und Datenstrukturen zur effizienten Sortierung großer Datenmengen", "Informatik - Algorithmen"),
    ("Maschinelle Lernverfahren für die Klassifikation von Bilddaten", "Informatik - KI/ML"),
    ("Statische und dynamische Analyse von Tragwerken aus Stahl und Beton", "Bauingenieurwesen"),
    ("Thermodynamische Prozesse in Verbrennungsmotoren und Wärmekraftmaschinen", "Maschinenbau"),
    ("Gleichstrom- und Wechselstromkreise mit komplexen Impedanzen", "Elektrotechnik"),
    ("Synthese und Charakterisierung von organischen Polymeren", "Chemie/Materialwissenschaft"),
    ("Mikroökonomische Marktmodelle und Preisbildungsmechanismen", "Wirtschaftswissenschaften")
]

for query, test_name in fachbereich_tests:
    test_query(query, test_name, n_results=5, show_details=False)


TEST: Informatik - Algorithmen
Query: Algorithmen und Datenstrukturen zur effizienten Sortierung großer Datenmengen

🔍 EMBEDDING-SUCHE: 3 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Algorithmen': 220 relevante Dokumente gefunden

TEST: Informatik - KI/ML
Query: Maschinelle Lernverfahren für die Klassifikation von Bilddaten

🔍 EMBEDDING-SUCHE: 5 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Maschinelle': 21 relevante Dokumente gefunden

TEST: Bauingenieurwesen
Query: Statische und dynamische Analyse von Tragwerken aus Stahl und Beton

🔍 EMBEDDING-SUCHE: 3 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Algorithmen': 220 relevante Dokumente gefunden

TEST: Informatik - KI/ML
Query: Maschinelle Lernverfahren für die Klassifikation von Bilddaten

🔍 EMBEDDING-SUCHE: 5 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Maschinelle': 21 relevante Dokumente gefunden

TEST: Bauingenieurwesen
Query: Statische und dynamische Analyse von Tragwerken aus Stahl und Beton

🔍 EMBEDDING-SUCHE: 5 relevan

### 2. Granularitäts-Tests (Spezifisch vs. Allgemein)

In [15]:
# Test verschiedener Granularitätsstufen
granularitaet_tests = [
    # Sehr spezifisch
    ("Implementierung des Quicksort-Algorithmus in Python mit Laufzeitanalyse", "Sehr spezifisch"),
    ("Quicksort Algorithmus", "Spezifisch"),
    ("Sortieralgorithmen", "Mittel"),
    ("Algorithmen", "Allgemein"),
    ("Informatik", "Sehr allgemein"),
    
    # Weiteres Beispiel
    ("Finite-Elemente-Methode zur Berechnung von Spannungsverteilungen in Stahlträgern", "Sehr spezifisch"),
    ("Finite Elemente", "Spezifisch"),
    ("Strukturanalyse", "Mittel"),
    ("Mechanik", "Allgemein")
]

for query, granularitaet in granularitaet_tests:
    test_query(query, f"Granularität: {granularitaet}", n_results=5, show_details=False)


TEST: Granularität: Sehr spezifisch
Query: Implementierung des Quicksort-Algorithmus in Python mit Laufzeitanalyse

🔍 EMBEDDING-SUCHE: 4 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Implementierung': 228 relevante Dokumente gefunden

TEST: Granularität: Spezifisch
Query: Quicksort Algorithmus

🔍 EMBEDDING-SUCHE: 1 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Quicksort': 11 relevante Dokumente gefunden

TEST: Granularität: Mittel
Query: Sortieralgorithmen

🔍 EMBEDDING-SUCHE: 5 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Sortieralgorithmen': 13 relevante Dokumente gefunden

TEST: Granularität: Allgemein
Query: Algorithmen

🔍 EMBEDDING-SUCHE: 4 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Implementierung': 228 relevante Dokumente gefunden

TEST: Granularität: Spezifisch
Query: Quicksort Algorithmus

🔍 EMBEDDING-SUCHE: 1 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Quicksort': 11 relevante Dokumente gefunden

TEST: Granularität: Mittel
Query: Sortieralgorithmen

🔍 EMBEDDING-SUC

### 3. Sprachstil-Tests (Fachsprache vs. Umgangssprache)

In [16]:
# Test verschiedener Sprachstile für das gleiche Konzept
sprachstil_tests = [
    # Transformatoren
    ("Transformatoren sind elektrische Betriebsmittel zur Spannungstransformation", "Fachsprache"),
    ("Wie funktioniert ein Trafo? Einfach erklärt für Anfänger", "Umgangssprache"),
    
    # Programmierung  
    ("Rekursive Algorithmen mit exponentieller Zeitkomplexität", "Fachsprache"),
    ("Wie programmiert man eine Funktion, die sich selbst aufruft?", "Umgangssprache"),
    
    # Mathematik
    ("Lineare Algebra: Eigenwerte und Eigenvektoren von Matrizen", "Fachsprache"),
    ("Mathe lernen: Was sind Eigenwerte? Verständlich erklärt", "Umgangssprache"),
    
    # Physik
    ("Elektromagnetische Induktion nach dem Faraday'schen Gesetz", "Fachsprache"),
    ("Warum entsteht Strom, wenn ich einen Magneten durch eine Spule bewege?", "Umgangssprache")
]

for query, stil in sprachstil_tests:
    test_query(query, f"Sprachstil: {stil}", n_results=3, show_details=False)


TEST: Sprachstil: Fachsprache
Query: Transformatoren sind elektrische Betriebsmittel zur Spannungstransformation

🔍 EMBEDDING-SUCHE: 3 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Transformatoren': 20 relevante Dokumente gefunden

TEST: Sprachstil: Umgangssprache
Query: Wie funktioniert ein Trafo? Einfach erklärt für Anfänger

🔍 EMBEDDING-SUCHE: 3 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Wie': 2102 relevante Dokumente gefunden

TEST: Sprachstil: Fachsprache
Query: Rekursive Algorithmen mit exponentieller Zeitkomplexität

🔍 EMBEDDING-SUCHE: 2 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Rekursive': 18 relevante Dokumente gefunden

TEST: Sprachstil: Umgangssprache
Query: Wie programmiert man eine Funktion, die sich selbst aufruft?

🔍 EMBEDDING-SUCHE: 3 relevante Dokumente gefunden

🔤 KEYWORD-SUCHE 'Wie': 2102 relevante Dokumente gefunden

TEST: Sprachstil: Fachsprache
Query: Rekursive Algorithmen mit exponentieller Zeitkomplexität

🔍 EMBEDDING-SUCHE: 2 relevante Dokumente gefu

### 4. Qualitätsbewertung und Relevanz-Analyse

In [17]:
# Detaillierte Analyse einer spezifischen Suchanfrage
def detailed_analysis(query_text, n_results=15):
    """
    Führt eine detaillierte Analyse einer Suchanfrage durch
    """
    print(f"\n🔍 DETAILLIERTE ANALYSE: '{query_text}'")
    print("="*80)
    
    # Embedding-Suche
    embedding = model.encode(query_text)
    results = collection.query(
        query_embeddings=[embedding],
        n_results=n_results
    )
    
    if not results["metadatas"] or not results["metadatas"][0]:
        print("❌ Keine Ergebnisse gefunden!")
        return
    
    # Erstelle detailliertes DataFrame
    detailed_df = pd.DataFrame(results["metadatas"][0])
    detailed_df['distance'] = results['distances'][0]  # Similarity score
    detailed_df['rank'] = range(1, len(detailed_df) + 1)
    
    # Merge mit AI-Metadaten
    detailed_df = detailed_df.merge(
        df_ai[['filename', 'ai:title', 'ai:revisedAuthor', 'ai:dewey', 'ai:keywords_ext']], 
        on='filename', how='left'
    )
    
    # Gruppiere nach Dokumenten
    doc_summary = detailed_df.groupby('filename').agg({
        'distance': 'min',  # Beste Similarity
        'page': 'nunique',  # Anzahl relevanter Seiten
        'ai:title': 'first',
        'ai:dewey': 'first',
        'ai:keywords_ext': 'first'
    }).sort_values('distance').reset_index()
    
    print(f"📊 ERGEBNISSE: {len(doc_summary)} relevante Dokumente gefunden")
    print(f"📄 Durchschnittlich {detailed_df.groupby('filename')['page'].nunique().mean():.1f} Seiten pro Dokument")
    
    # Top 5 Dokumente anzeigen
    print(f"\n🏆 TOP 5 RELEVANTE DOKUMENTE:")
    for idx, row in doc_summary.head(5).iterrows():
        print(f"  {idx+1}. {row['ai:title'][:60]}...")
        print(f"     📐 Similarity: {row['distance']:.3f} | 📚 DDC: {row['ai:dewey']} | 📄 Seiten: {row['page']}")
        if pd.notna(row['ai:keywords_ext']):
            keywords = str(row['ai:keywords_ext'])[:100] + "..." if len(str(row['ai:keywords_ext'])) > 100 else str(row['ai:keywords_ext'])
            print(f"     🏷️  Keywords: {keywords}")
        print()
    
    # DDC-Verteilung
    ddc_dist = doc_summary['ai:dewey'].value_counts().head(5)
    print(f"📊 DDC-KATEGORIEN (Top 5):")
    for ddc, count in ddc_dist.items():
        print(f"  {ddc}: {count} Dokumente")
    
    return doc_summary

# Beispiel-Analyse
detailed_analysis("Machine Learning Algorithmen für Bilderkennung und Computer Vision")


🔍 DETAILLIERTE ANALYSE: 'Machine Learning Algorithmen für Bilderkennung und Computer Vision'
📊 ERGEBNISSE: 10 relevante Dokumente gefunden
📄 Durchschnittlich 1.5 Seiten pro Dokument

🏆 TOP 5 RELEVANTE DOKUMENTE:
  1. Basics of Bibliographic Cataloging and Indexing...
     📐 Similarity: 15.161 | 📚 DDC:  | 📄 Seiten: 2
     🏷️  Keywords: Wahrnehmungspsychologie, Kognitive Psychologie, visuelle Wahrnehmung, Gesichtsverarbeitung, Reaktion...

  2. Statistik mit R...
     📐 Similarity: 16.003 | 📚 DDC:  | 📄 Seiten: 2
     🏷️  Keywords: Zeitreihe, Wahrscheinlichkeitsverteilung, Quantile, Normalverteilung, lineares Modell, exponentielle...

  3. Statistik mit R...
     📐 Similarity: 16.003 | 📚 DDC:  | 📄 Seiten: 2
     🏷️  Keywords: Zeitreihenanalyse, Datenanalyse, Statistik, lineare Modelle, Regression, Indexierung, Bibliothekaris...

  4. Gestalttheorie und kognitive Psychologie...
     📐 Similarity: 16.338 | 📚 DDC:  | 📄 Seiten: 1
     🏷️  Keywords: Gestaltpsychologie, Wahrnehmungsqualitäten,

,filename,distance,page,ai:title,ai:dewey,ai:keywords_ext
0,10UZ15z9_nbqo.pdf,15.161404,2,Basics of Bibliographic Cataloging and Indexing,,"Wahrnehmungspsychologie, Kognitive Psychologie..."
1,11CO2XNi2cEJs.pdf,16.002508,2,Statistik mit R,,"Zeitreihe, Wahrscheinlichkeitsverteilung, Quan..."
2,7sEuVkT6B6Is.pdf,16.002508,2,Statistik mit R,,"Zeitreihenanalyse, Datenanalyse, Statistik, li..."
3,10JW63ui75ahk.pdf,16.337620,1,Gestalttheorie und kognitive Psychologie,,"Gestaltpsychologie, Wahrnehmungsqualitäten, Le..."
4,10w0l3DCqcr4I.pdf,16.408207,1,Kognitive Architektur und Problemlösendes Denken,,"Somatische Marker, Kognitionswissenschaften, E..."
5,10u2FLKt0ieZs.pdf,16.449423,3,Assoziatives Lernen,,"Assoziation, Konditionierung, Beobachtungslern..."
6,119yJIgL-jkro.pdf,16.576340,1,Aufgaben zur Vorlesung Algorithmen und Datenst...,,"Bedienungsprogramm, Bibliothekarische Erschlie..."
7,3wPcrjpcGtFg.pdf,16.638172,1,Methoden der empirischen Sozialforschung,,"Phänomenologie, Sozialforschung, Lebenswelt, I..."
8,3uDbsS7QvJ64.pdf,16.682760,1,Machine Learning I PLTSVT,,"Maschinelles Lernen, Künstliche Intelligenz, D..."
9,67asiEvWQUNE.pdf,16.758703,1,Ein Beispiel für das Potenzial von Large Langu...,,"Grenzwert, Untervektorraum, Reelle Funktion, M..."
